geoDist now has cleaner columns, daily deaths per 1000, and daily cases per 1000, as well as the continent

In [40]:
%%bigquery
SELECT *
FROM covid_modeled.geoDist
LIMIT 5

,code,country,date,daily_cases,daily_deaths,cases,deaths,deaths_per_100000,cases_per_100000,pop,continent
0,AF,Afghanistan,2019-12-31,0,0,0,0,0.0,0.0,37172386,Asia
1,DZ,Algeria,2019-12-31,0,0,0,0,0.0,0.0,42228429,Africa
2,AM,Armenia,2019-12-31,0,0,0,0,0.0,0.0,2951776,Asia
3,AU,Australia,2019-12-31,0,0,0,0,0.0,0.0,24992369,Oceania
4,AT,Austria,2019-12-31,0,0,0,0,0.0,0.0,8847037,Europe


mobility now has a composite primary key of date and country/code. All countries in mobility are in geoDist. The average change column gives the average across all six locations.

In [51]:
%%bigquery
SELECT * 
FROM covid_modeled.mobility
LIMIT 5

,code,country,date,average_change,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,AF,Afghanistan,2020-02-15,-0.07,-0.09,-0.10,0.00,-0.02,-0.28,0.05
1,AO,Angola,2020-02-15,0.02,0.03,-0.02,0.07,0.03,0.02,0.01
2,AG,Antigua and Barbuda,2020-02-15,-0.02,-0.06,-0.01,-0.06,-0.01,0.00,0.03
3,AR,Argentina,2020-02-15,-0.02,0.00,-0.02,-0.06,-0.05,-0.01,0.01
4,AW,Aruba,2020-02-15,0.06,0.02,0.03,0.12,0.26,-0.02,-0.02


Look at the last day that data is available in both tables

In [17]:
%%bigquery
SELECT MAX(date)
FROM covid_modeled.mobility

,f0_
0,2020-04-30


Look at countries where covid has been the deadliest. (Most deaths)

In [41]:
%%bigquery
SELECT * 
FROM covid_modeled.geoDist
WHERE date = '2020-04-30'
ORDER BY deaths DESC
LIMIT 10

,code,country,date,daily_cases,daily_deaths,cases,deaths,deaths_per_100000,cases_per_100000,pop,continent
0,US,United_States_of_America,2020-04-30,27326,2611,1039909,60966,0.79806,8.35230,327167434,North America
1,IT,Italy,2020-04-30,2086,323,203591,27682,0.53449,3.45185,60431283,Europe
2,UK,United_Kingdom,2020-04-30,4076,795,165221,26097,1.19569,6.13034,66488991,Europe
3,ES,Spain,2020-04-30,518,268,213435,24543,0.57358,1.10864,46723749,Europe
4,FR,France,2020-04-30,1607,427,128442,24087,0.63743,2.39896,66987244,Europe
5,BE,Belgium,2020-04-30,525,170,47859,7501,1.48835,4.59637,11422068,Europe
6,DE,Germany,2020-04-30,1478,173,159119,6288,0.20861,1.78227,82927922,Europe
7,IR,Iran,2020-04-30,1073,80,93657,5957,0.09780,1.31173,81800269,Asia
8,BR,Brazil,2020-04-30,6276,449,78162,5466,0.21435,2.99614,209469333,South America
9,NL,Netherlands,2020-04-30,386,145,38802,4711,0.84151,2.24015,17231017,Europe


Where have there been the most cases?

In [47]:
%%bigquery
SELECT * 
FROM covid_modeled.geoDist
WHERE date = '2020-04-30'
ORDER BY cases DESC
LIMIT 10

,code,country,date,daily_cases,daily_deaths,cases,deaths,deaths_per_100000,cases_per_100000,pop,continent
0,US,United_States_of_America,2020-04-30,27326,2611,1039909,60966,0.79806,8.35230,327167434,North America
1,ES,Spain,2020-04-30,518,268,213435,24543,0.57358,1.10864,46723749,Europe
2,IT,Italy,2020-04-30,2086,323,203591,27682,0.53449,3.45185,60431283,Europe
3,UK,United_Kingdom,2020-04-30,4076,795,165221,26097,1.19569,6.13034,66488991,Europe
4,DE,Germany,2020-04-30,1478,173,159119,6288,0.20861,1.78227,82927922,Europe
5,FR,France,2020-04-30,1607,427,128442,24087,0.63743,2.39896,66987244,Europe
6,TR,Turkey,2020-04-30,2936,89,117589,3081,0.10812,3.56658,82319724,Europe
7,RU,Russia,2020-04-30,5841,105,99399,972,0.07268,4.04283,144478050,Europe
8,IR,Iran,2020-04-30,1073,80,93657,5957,0.09780,1.31173,81800269,Asia
9,CN,China,2020-04-30,4,0,83944,4637,0.00000,0.00029,1392730000,Asia


Look at changes in mobility versus Daily cases in the US

In [49]:
%%bigquery
SELECT g.code, g.date, daily_cases, average_change, retail_and_recreation, grocery_and_pharmacy, parks, transit_stations, 
workplaces, residential
FROM `corvid-276516.covid_modeled.geoDist` g
JOIN `corvid-276516.covid_modeled.mobility` m
ON m.code = g.code 
AND m.date = g.date
WHERE g.code = "US"
ORDER BY date

,code,date,daily_cases,average_change,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,US,2020-02-15,0,0.04,0.06,0.02,0.17,0.04,0.00,-0.01
1,US,2020-02-16,0,0.04,0.08,0.03,0.18,0.04,0.00,-0.01
2,US,2020-02-17,0,-0.03,0.04,-0.00,0.28,0.02,-0.18,0.04
3,US,2020-02-18,0,0.00,-0.00,-0.01,0.05,0.02,-0.00,0.01
4,US,2020-02-19,0,0.01,0.02,0.01,0.08,0.02,0.01,0.00
...,...,...,...,...,...,...,...,...,...,...
71,US,2020-04-26,48529,-0.22,-0.32,-0.11,0.05,-0.31,-0.30,0.10
72,US,2020-04-27,26857,-0.21,-0.25,-0.07,0.08,-0.26,-0.35,0.15
73,US,2020-04-28,22541,-0.22,-0.24,-0.05,0.21,-0.23,-0.37,0.10
74,US,2020-04-29,24132,-0.22,-0.24,-0.04,0.08,-0.23,-0.37,0.16


In [54]:
%%bigquery
CREATE OR REPLACE VIEW covid_modeled.v__US_covidDailyVmobility AS
SELECT g.code, g.date, daily_cases, deaths, average_change, retail_and_recreation, grocery_and_pharmacy, parks, transit_stations, 
workplaces, residential
FROM `corvid-276516.covid_modeled.geoDist` g
JOIN `corvid-276516.covid_modeled.mobility` m
ON m.code = g.code 
AND m.date = g.date
WHERE g.code = "US"
ORDER BY date

""


Look at countries rated by percentile values

In [23]:
%%bigquery
SELECT country, pop, percentile
FROM
  (SELECT country,pop, ROUND(PERCENT_RANK() OVER(ORDER BY pop),2) AS percentile
  FROM
    (SELECT DISTINCT country, code, pop
     FROM covid_modeled.geoDist))
WHERE percentile >= .7
ORDER BY percentile DESC
LIMIT 60

,country,pop,percentile
0,China,1392730000,1.00
1,India,1352617328,1.00
2,Indonesia,267663435,0.99
3,United_States_of_America,327167434,0.99
4,Pakistan,212215030,0.98
5,Brazil,209469333,0.98
6,Bangladesh,161356039,0.97
7,Nigeria,195874740,0.97
8,Russia,144478050,0.96
9,Japan,126529100,0.96


Using countries in the top .85 percentile and average cases per 100,000, it can be examined which countries have handled the pandemic the best, despite having a large population, and if there is a correlation between countries that have had a large reduction in mobility, and average daily cases per 100,000.

In [55]:
%%bigquery
SELECT g.country, ROUND(AVG(average_change),2) AS average_change,ROUND(AVG(cases_per_100000),2) AS avg_cases_per_100000, MAX(pop) AS pop
FROM `corvid-276516.covid_modeled.mobility` m
JOIN `corvid-276516.covid_modeled.geoDist` g 
ON g.code = m.code
WHERE g.country IN (SELECT country
 FROM 
 (SELECT country,pop, PERCENT_RANK() OVER(ORDER BY pop) AS percentile
  FROM
    (SELECT DISTINCT country, code, pop
     FROM covid_modeled.geoDist))
  WHERE percentile >= .95)
GROUP BY country

,country,average_change,avg_cases_per_100000,pop
0,Bangladesh,-0.20,0.13,161356039
1,Brazil,-0.19,0.43,209469333
2,China,-0.13,0.05,1392730000
3,India,-0.20,0.03,1352617328
4,Indonesia,-0.14,0.04,267663435
5,Japan,-0.05,0.09,126529100
6,Mexico,-0.17,0.17,126190788
7,Nigeria,-0.11,0.01,195874740
8,Pakistan,-0.19,0.09,212215030
9,United_States_of_America,-0.11,2.88,327167434


In [52]:
%%bigquery
CREATE OR REPLACE VIEW covid_modeled.v__changeVcases_in_lg_countries AS
SELECT m.country, ROUND(AVG(average_change),2) AS average_change, ROUND(AVG(cases_per_100000),2) AS avg_cases_per_100000, MAX(pop) AS pop
FROM `corvid-276516.covid_modeled.mobility` m
JOIN `corvid-276516.covid_modeled.geoDist` g 
ON g.code = m.code
WHERE g.country IN (SELECT country
 FROM 
 (SELECT country,pop, PERCENT_RANK() OVER(ORDER BY pop) AS percentile
  FROM
    (SELECT DISTINCT country, code, pop
     FROM `corvid-276516.covid_modeled.geoDist`))
WHERE percentile >= .85)
GROUP BY m.country

""


India has a much larger population than America, however, it has expereienced far fewer daily cases per 1000 on average. Focus in on mobility changes in India with respect to daily cases.

In [67]:
%%bigquery
CREATE OR REPLACE VIEW covid_modeled.v__India_covidDailyVmobility AS
SELECT g.code, g.date, daily_cases, deaths, average_change, retail_and_recreation, grocery_and_pharmacy, parks, transit_stations, workplaces
FROM `corvid-276516.covid_modeled.geoDist` g
JOIN `corvid-276516.covid_modeled.mobility` m
ON m.code = g.code 
AND m.date = g.date
WHERE g.code = "IN"
ORDER BY date

""


Look at mobility changes and daily cases per 100,000 by continent and date

In [53]:
%%bigquery
SELECT continent, m.date, ROUND(SUM(daily_cases)/SUM(pop/100000),2) AS avg_cases, ROUND(AVG(average_change),2) AS avg_change, 
COUNT(DISTINCT m.code) AS num_countries
FROM `corvid-276516.covid_modeled.mobility` m
JOIN `corvid-276516.covid_modeled.geoDist`g
ON m.code = g.code
AND m.date = g.date
GROUP BY continent, m.date

,continent,date,avg_cases,avg_change,num_countries
0,Asia,2020-02-15,0.07,0.01,25
1,Oceania,2020-02-15,0.00,0.02,2
2,Europe,2020-02-15,0.00,0.05,23
3,South America,2020-02-15,0.00,0.01,2
4,North America,2020-02-15,0.00,0.02,4
...,...,...,...,...,...
450,Africa,2020-04-30,0.15,-0.22,27
451,North America,2020-04-30,5.35,-0.34,19
452,South America,2020-04-30,2.48,-0.39,10
453,Oceania,2020-04-30,0.03,-0.23,4


In [54]:
%%bigquery
CREATE OR REPLACE VIEW covid_modeled.v__mobCases_cont AS
SELECT continent, m.date, ROUND(SUM(daily_cases)/SUM(pop/100000),2) AS avg_cases, ROUND(AVG(average_change),2) AS avg_change, 
COUNT(DISTINCT m.code) AS num_countries
FROM `corvid-276516.covid_modeled.mobility` m
JOIN `corvid-276516.covid_modeled.geoDist`g
ON m.code = g.code
AND m.date = g.date
GROUP BY continent, m.date

""


Mobility change by date for the entire world

In [19]:
%%bigquery
CREATE OR REPLACE VIEW covid_modeled.v__world_mobility AS
SELECT date, ROUND(AVG(average_change), 2) AS world_avg_change 
FROM `corvid-276516.covid_modeled.mobility` 
GROUP BY date

""
